In [ ]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [128]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-25 07:41:21,639 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-25 07:41:21,641 - settings - Setting database.user to celiib
INFO - 2021-02-25 07:41:21,642 - settings - Setting database.password to newceliipass
INFO - 2021-02-25 07:41:21,648 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-25 07:41:21,649 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:21,664 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-25 07:41:22,071 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:22,111 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-25 07:41:22,514 - settings - Setting enable_python_native_blobs to True


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload
import trimesh_utils as tu

# Getting a more high fidelity axon skeleton

In [51]:
minnie.AutoProofreadValidationNeurons() & "axon_length>300"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135113167769,0,0.25,30.00,5,0,0,2,1,excitatory,densely_spined,1,1,1521.4480575529685,1676.4575752748142,335175,917.6,1,1,198908,188769,19783

In [5]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [62]:
segment_id,split_index = 864691135426651890,0 #long axon good for proofreading
#segment_id, split_index = 864691135446640468,0 #comes off the side of dendrite
segment_id,split_index = 864691135699269154,0
segment_id,split_index = 864691135589907979,0
segment_id, split_index = 864691135735426708,0

In [63]:
validation = True

In [64]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True,
                        validation=True)

Synapsse Results:
Total Number of Synapses = 3713
n_valid_syn_centers_presyn = 67
n_errored_syn_centers_presyn = 193
n_valid_syn_centers_postsyn = 2830
n_errored_syn_centers_postsyn = 623
Including axon mesh


In [65]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Dont need spine recalculation
Number of Neurons found = 1


In [66]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [67]:
neuron_obj.segment_id

864691135735426708

In [68]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [69]:
clu.axon_classification(neuron_obj,
                        plot_candidates=True,
                       best_axon=True,
                       plot_axons=True)


Part 1: Axon like branchese 
{'L0': array([ 4,  6, 17, 23, 26, 27]), 'L1': array([ 5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]), 'L2': array([0, 1, 2]), 'L6': array([0, 3, 4, 6, 7, 8])}
Limb L0 soma angle: 16.5 
Limb L1 soma angle: 165.42 
Limb L2 soma angle: 95.54 
Limb L3 soma angle: 95.59 
Limb L4 soma angle: 136.45 
Limb L5 soma angle: 119.1 
Limb L6 soma angle: 87.88 

Part 2: possible_axon_limbs = [1, 2, 3, 4, 5, 6]

Phase 2: Working on Limb L1
nodes_to_eliminate = []

Phase 2: Working on Limb L2
nodes_to_eliminate = [5 7]

Phase 2: Working on Limb L3
nodes_to_eliminate = [4]

Phase 2: Working on Limb L4
nodes_to_eliminate = [2]

Phase 2: Working on Limb L5
nodes_to_eliminate = []

Phase 2: Working on Limb L6
nodes_to_eliminate = []

----Working on Limb L1-----
No nodes were eliminated so don't need to add back any candi

Final Candidates

Limb 1
Candidate 0 (cyan): array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53])

Limb 2
Candidate 0 (cadetblue): array([10, 11,  6])
Candidate 1 (gold): array([8])
Candidate 2 (brown): array([ 0,  1,  2,  3,  4,  9, 12])

Limb 3
Candidate 0 (magenta): array([0])
Candidate 1 (blue): array([1, 2, 3])

Limb 4
Candidate 0 (cadetblue): array([0, 1, 3])
Candidate 1 (olive): array([4])

Limb 5
Candidate 0 (gold): array([0, 1, 2])

Limb 6
Candidate 0 (tan): array([0, 1, 2, 3, 4, 5, 6, 7, 8])
Part 3: axon_subgraph_candidates = {1: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]], 2: [[10, 11, 6], [8], [0, 1, 2, 3, 4, 9, 12]], 3: [[

({'L1': array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
         13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
         26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
         39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
         52., 53.])},
 {'L0': array([ 4, 17, 23, 26, 27]),
  'L2': array([0, 1, 2]),
  'L6': array([0, 3, 4, 6, 7, 8])})

# Erasing the starting branch from the axon so can test if axon is not starter

In [70]:
# axon_limb_branch_dict= neuron_obj.axon_limb_branch_dict

# axon_limb_name = neuron_obj.axon_limb_name
# curr_limb = neuron_obj[axon_limb_name]
# axon_branches = axon_limb_branch_dict[axon_limb_name]
# curr_limb.current_starting_node,neuron_obj.axon_starting_branch

# neuron_obj[axon_limb_name][neuron_obj.axon_starting_branch].labels = []

In [71]:
nviz.plot_axon(neuron_obj)


 Working on visualization type: mesh


# The whole process of replacing the axon with higher fidelity

In [85]:
new_neuron_obj = pru.refine_axon_for_high_fidelity_skeleton(neuron_obj=neuron_obj,
    plot_new_axon_limb_correspondence = True,
    plot_new_limb_object = True,
    plot_final_revised_axon_branch=True,
     verbose = True,)



Starting axon branch was the starting node so using border vertices and NOT computing upstream branch
upstream_node_to_axon_starting_branch = None
root_curr = [1348840.   541246.2  817276.4]
Time for preparing soma vertices and root: 0.001306295394897461
invalidation_d = 2000
cc_vertex_thresh = 10


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:06<00:00,  6.02s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 7.221849203109741
branches_touching_root = [37]
combining close nodes
combine_threshold = 1300
length of Graph = 29071
Working on path [1555. 1562. 1569. 1578. 1582. 1584. 1588. 1592. 1594. 1597. 1599. 1600.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
Working on path [3428. 3431. 3435. 3438. 3441. 3444. 3446. 3449. 3451. 3452. 3455.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
Working on path [3776. 3779. 3786. 3797. 3805. 3813. 3819.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [4007. 4013. 4015. 4017. 4019. 4021. 4025. 4037.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [4841. 4845. 4848. 4851. 4854. 4856. 4862.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [12243. 12255. 12267. 12280. 12291. 12307. 12319. 12336. 12355. 12374.
 12398. 12423. 12428.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
Working on path [13403. 13422. 13440. 13463. 13473. 13480. 13486. 13494.]



AFTER face_lookup_resolved_test


...


250708 250709
For marked faces: None
empty_indices % = 0.00035499323917370337
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Upstream node was None so don't have to adjust
axon_limb_name= L1
Limb starting node = 45
Axon starting node = 45
axon_branches = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53]
Using Axon Starting Info
Leftover Nodes After Axon Deletion = []
New Node names = []
Leftover Nodes Mapping = {}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7efdeb5cf1d0>]}
New Limb Object


Working on limb L1 branch 132
Working on limb L1 branch 123
Working on limb L1 branch 134
Working on limb L1 branch 99
Working on limb L1 branch 124
Working on limb L1 branch 127
Working on limb L1 branch 133
Working on limb L1 branch 90
Working on limb L1 branch 100
Working on limb L1 branch 138
Working on limb L1 branch 140
Working on limb L1 branch 141
Working on limb L1 branch 71
Working on limb L1 branch 91
Working on limb L1 branch 106
Working on limb L1 branch 113
Working on limb L1 branch 52
Working on limb L1 branch 70
Working on limb L1 branch 111
Working on limb L1 branch 116
Working on limb L1 branch 87
Working on limb L1 branch 104
Working on limb L1 branch 97
Working on limb L1 branch 114
Working on limb L1 branch 73
Working on limb L1 branch 74
Working on limb L1 branch 50
Working on limb L1 branch 105
Working on limb L1 branch 85
Working on limb L1 branch 92
Working on limb L1 branch 96
Working on limb L1 branch 98
Working on limb L1 branch 119
Working on limb L1 branch



---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1348964.14024777  541728.93462084  817404.77605041]---------
Starting_edge inside branches_to_conept = [[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [123]
printing out current edge:
[[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
edge_endpoints_to_process was empty so exiting loop after 163 iterations
starting_node in concept map (that should match the starting edge) = 132
Total time for branches to concept conversion = 1.3947954177856445

Done generating concept network 



 Working on visualization type: mesh

 Working on visualization type: skeleton


In [86]:
import neuron_searching as ns
ns.query_neuron_by_labels(new_neuron_obj,matching_labels=["axon"])

{'L1': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162])}

In [87]:
nviz.plot_axon(new_neuron_obj)


 Working on visualization type: mesh


# Try proofreading the neuron (using new rules for high fidelity)

In [75]:
pru.get_excitatory_high_fidelity_axon_pre_filters()

[{'filter_name': 'dendrite_on_axon_merges',
  'filter_function': <function proofreading_utils.filter_away_dendrite_on_axon_merges(neuron_obj, perform_deepcopy=True, limb_branch_dict_for_search=None, use_pre_existing_axon_labels=False, perform_axon_classification=False, dendritic_merge_on_axon_query=None, dendrite_merge_skeletal_length_min=20000, dendrite_merge_width_min=100, dendritie_spine_density_min=0.00015, plot_limb_branch_filter_away=False, plot_limb_branch_filter_with_disconnect_effect=False, return_error_info=False, plot_final_neuron=False, verbose=False)>,
  'filter_kwargs': {'use_pre_existing_axon_labels': True}}]

In [88]:
plot_limb_branch_filter_with_disconnect_effect = True
verbose = False
verbose_outline = True
o_neuron, filtering_info = pru.apply_proofreading_filters_to_neuron(input_neuron = neuron_obj,
                                            filter_list = pru.get_excitatory_high_fidelity_axon_pre_filters(),
                        plot_limb_branch_filter_with_disconnect_effect=plot_limb_branch_filter_with_disconnect_effect,
                                            plot_limb_branch_filter_away=False,
                                            plot_final_neuron=False,

                                            return_error_info=True,
                                             verbose=verbose,
                                            verbose_outline=verbose_outline
        )


--- Working on filter 0:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh





 ---- Total time for applying filter: 24.853794813156128 -----


In [89]:
filtering_info

{'dendrite_on_axon_merges_neuron': <neuron.Neuron at 0x7efcfbf4dc50>,
 'dendrite_on_axon_merges_time': 24.853686571121216,
 'dendrite_on_axon_merges_error_area': 633.8468374430994,
 'dendrite_on_axon_merges_error_length': 271.456668943723}

In [81]:
new_neuron_obj.axon_limb_branch_dict

{}

In [90]:
nviz.plot_axon(o_neuron)

[autoreload of proofreading_utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/meshAfterParty/meshAfterParty/proofreading_utils.py", line 3765
    else:
 


 Working on visualization type: mesh


In [94]:
nviz.visualize_neuron_lite(neuron_obj)


 Working on visualization type: mesh


In [126]:
output_dict= pru.proofread_neuron(neuron_obj,
                            plot_limb_branch_filter_with_disconnect_effect=False,
                            plot_final_filtered_neuron=False,
                            verbose=True)

filtered_neuron = output_dict["filtered_neuron"]
cell_type_info = output_dict["cell_type_info"]
filtering_info = output_dict["filtering_info"]

---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


base_final_widths = [428.64984057 437.9965134 ]
overall_ais_width = 330.8527724540625
ais_width = 433.3231769814422
Axon Classification = 27.649645566940308


 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
Did not pass final filters to continuing
Inhibitory Excitatory Classification = 10.288856029510498

 -- Cell Type Classification Results --
inh_exc_class=excitatory
spine_category=densely_spined
axon_angles={1: {0: 167.3683054842977}}
n_axons=1
n_apicals=1
neuron_spine_density=0.0006585676303633324
n_branches_processed=25
skeletal_length_processed=2195870.1855782047
n_branches_in_search_radius=51
skeletal_length_in_search_radius=2683849.2278394564


 ------ Part D: Neuron Filtering ---- 

 


 Using high_fidelity_axon_on_excitatory


---Step 1: Applying Dendrite on Axon Filtering
INSIDE THE NEW EXCITATORY FILTERS

  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [1348840.   541246.2  817276.4]
Time for preparing soma vertices and root: 0.0018048286437988281
invalidation_d = 2000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:04<00:00,  4.20s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 5.32108736038208
branches_touching_root = [17]
combining close nodes
combine_threshold = 1300
length of Graph = 26007
After combining close endpoints max(kept_branches_idx) = 173, len(kept_branches_idx) = 163
empty_indices % = 0.0
 conflict_indices % = 0.01873853562546661


...


202426 202427
For marked faces: None
empty_indices % = 0.0004643649315555731
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Upstream node was None so don't have to adjust
axon_limb_name= L1
Limb starting node = 41
Axon starting node = 41
axon_branches = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
Using Axon Starting Info
Leftover Nodes After Axon Deletion = []
New Node names = []
Leftover Nodes Mapping = {}
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7efdfd68d4a8>]}
Working on limb L1 branch 117
Working on limb L1 branch 108
Working on limb L1 branch 119
Working on limb L1 branch 84
Working on limb L1 branch 109
Working on limb L1 branch 112
Working on limb L1 branch 118
Working on limb L1 branch 75
Working on limb L1 branch 85
Working on limb L1 branch 123
Working on limb L1 branch 125
Working on limb L1 branch 126
Working on limb L1 branch 56
Working on limb L1 branch 76
Working on limb L1 branch 91
Working on limb L1 branch 98
Working on limb L1 branch 37
Working on



---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1348964.14024777  541728.93462084  817404.77605041]---------
Starting_edge inside branches_to_conept = [[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [108]
printing out current edge:
[[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
edge_endpoints_to_process was empty so exiting loop after 148 iterations
starting_node in concept map (that should match the starting edge) = 117
Total time for branches to concept conversion = 1.4812805652618408

Done generating concept network 




---- Step 3: Applying Excitatory Filters Post-processing -----

--- Working on filter 0:
function = axon_on_dendrite_merges
function __name__ = filter_away_axon_on_dendrite_merges
function arguments = {'use_pre_existing_axon_labels': T

----------------------- FINISHED Running Filter ------------------



 --Filter double_back_and_width_change Results --
local_results = {'double_back_and_width_change_neuron': <neuron.Neuron object at 0x7efdec8afdd8>, 'double_back_and_width_change_time': 34.199880838394165, 'double_back_and_width_change_error_area': 100.65707498638605, 'double_back_and_width_change_error_length': 150.63852762626743}

--- Working on filter 4:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True, 'min_degree_to_find': 4}


----------------------- Running Filter ------------------
min_degree_to_find = 4
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L0 ---
edges_to_create = []
edges_to_create = []
min_degree_to_find = 4
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L1 ---
edges_t

In [127]:
nviz.visualize_neuron(filtered_neuron,
                     limb_branch_dict={filtered_neuron.axon_limb_name:"all"})


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Development

In [125]:
o_neuron,filtering_info = pru.proofread_neuron_class_predetermined(neuron_obj=neuron_obj,
            inh_exc_class = "excitatory",
            plot_limb_branch_filter_with_disconnect_effect = True,
            verbose = False,
            verbose_outline = True,
            high_fidelity_axon_on_excitatory = True,
            plot_final_filtered_neuron = False,)



 Using high_fidelity_axon_on_excitatory


---Step 1: Applying Dendrite on Axon Filtering
INSIDE THE NEW EXCITATORY FILTERS: {'filter_name': 'crossovers', 'filter_function': <function filter_away_crossovers at 0x7efdeccc17b8>, 'filter_kwargs': {'axon_dependent': True, 'match_threshold': 50, 'require_two_pairs': False}}

--- Working on filter 0:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh





 ---- Total time for applying filter: 25.799861669540405 -----


---- Step 2: Applying High Fidelity Axon -----
Starting axon branch was the starting node so using border vertices and NOT computing upstream branch
upstream_node_to_axon_starting_branch = None


  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [1348840.   541246.2  817276.4]
Time for preparing soma vertices and root: 0.007697105407714844
invalidation_d = 2000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:04<00:00,  4.82s/it]

...


AFTER face_lookup_resolved_test



Upstream node was None so don't have to adjust
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7efcd357b6d8>]}
Assigning the old width calculation because no valid new widths


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1348964.14024777  541728.93462084  817404.77605041]---------
Starting_edge inside branches_to_conept = [[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [108]
printing out current edge:
[[1348964.14024777  541728.93462084  817404.77605041]
 [1351517.84565173  560608.21852905  821071.11405119]]
edge_endpoints_to_process was empty so exiting loop after 148 iterations
starting_node in concept map (that should match the starting edge) = 117
Total time for branches to concept conversion = 1.507692575454712

Done generating concept network 



--- Working on filter 1:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 50, 'require_two_pairs': False}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 2:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': True, 'skip_double_back_errors_for_axon': False, 'width_jump_threshold': 250, 'running_width_jump_method': True, 'double_back_axon_like_threshold': 145, 'double_back_threshold': 120}




Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh



--- Working on filter 4:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True, 'min_degree_to_find': 4}


min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4



 ---- Total time for applying filter: 82.70706129074097 -----


# Analyzing how the crossover did

In [124]:
after_crossover_neuron = filtering_info["crossovers_neuron"]

axon_limb_name = after_crossover_neuron.axon_limb_name

high_degree_coords = nru.high_degree_branching_coordinates_on_limb(after_crossover_neuron[axon_limb_name],
                                             min_degree_to_find=4)

nviz.visualize_neuron(after_crossover_neuron,limb_branch_dict={axon_limb_name:"all"},
                     scatters=[high_degree_coords],
                     scatter_size=0.1)


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [117]:
high_degree_coords = nru.high_degree_branching_coordinates_on_limb(after_crossover_neuron[axon_limb_name],
                                             min_degree_to_find=5)

nviz.visualize_neuron(after_crossover_neuron,limb_branch_dict={axon_limb_name:"all"},
                     scatters=[high_degree_coords],
                     scatter_size=0.2)


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [118]:
high_degree_coords

array([[1334595.81146576,  658912.32150835,  847548.99412118],
       [1338068.99895845,  648534.46800445,  847560.49040373]])

In [109]:
nviz.visualize_limb_obj(after_crossover_neuron[0])

In [111]:
after_crossover_neuron.axon_limb_name

'L1'

In [110]:
nviz.visualize_limb_obj(after_crossover_neuron["L0"])